In [2]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Assign the database variable
db = mongo.divvy_db

# Assign the colloections variable
divvy_rides = db['divvy_ride_data']
weather_daily = db['weather_daily']
divvy_ride_data_merged = db["divvy_ride_data_merged"]

print(f'Mongo Database Name: {db.name}')
print(f'Mongo Collection Names: {db.list_collection_names()}')

Mongo Database Name: divvy_db
Mongo Collection Names: ['divvy_ride_data', 'divvy_ridedata', 'weather_daily']


In [4]:
# Show all the collections with docoment count
collection_names = db.list_collection_names()

for collection_name in collection_names:
    collection = db[collection_name]
    document_count = collection.count_documents({})
    print(f'Collection: {collection_name}, Document Count: {document_count}')

Collection: divvy_ride_data, Document Count: 5667717
Collection: divvy_ridedata, Document Count: 0
Collection: weather_daily, Document Count: 365


In [17]:
document = divvy_rides.find_one()
pprint(document)

{'_id': ObjectId('650f7a7291cca7bfdb2c89da'),
 'end_lat': 42.01256011541,
 'end_lng': -87.6743671152,
 'end_station_id': 'RP-007',
 'end_station_name': 'Clark St & Touhy Ave',
 'ended_at': '2022-01-10 08:46:17',
 'member_casual': 'casual',
 'ride_id': 'A6CF8980A652D272',
 'rideable_type': 'electric_bike',
 'start_lat': 42.012763,
 'start_lng': -87.6659675,
 'start_station_id': 525,
 'start_station_name': 'Glenwood Ave & Touhy Ave',
 'started_at': '2022-01-10 08:41:56'}


In [10]:
divvy_rides.create_index([("started_at_date", 1)])
weather_daily.create_index([("date", 1)])

'date_1'

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'weather_daily',
            'localField': 'started_at_date',
            'foreignField': 'date',
            'as': 'weather_data'
        }
    },
    {
        '$unwind': {
            'path': '$weather_data',
            'preserveNullAndEmptyArrays': True
        }
    },
    {
        '$merge': {
            'into': 'divvy_ride_data_merged',  # Replace with your new collection name
            'whenMatched': 'merge',  # Merge documents with matching _id fields
            'whenNotMatched': 'insert'  # Insert documents that don't match
        }
    },
]

divvy_rides.aggregate(pipeline)
print("Update completed successfully.")

In [ ]:
# Access the database and collection
collection = db["divvy_ride_data_merged"]  # Replace with your collection name

# Count the documents in the collection
document_count = collection.count_documents({})
print("Total number of documents:", document_count)